In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
from sklearn.preprocessing import LabelEncoder
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import mean_squared_error, r2_score


# Load the LCL dataset

In [ ]:
import os

folder_path = '/content/drive/MyDrive/Low Carbon dataset'
file_list = os.listdir(folder_path)
csv_files = [f for f in file_list if f.endswith('.csv')]

print(csv_files)


['LCL-June2015v2_0.csv', 'LCL-June2015v2_1.csv', 'LCL-June2015v2_2.csv', 'LCL-June2015v2_3.csv', 'LCL-June2015v2_4.csv', 'LCL-June2015v2_5.csv', 'LCL-June2015v2_6.csv', 'LCL-June2015v2_7.csv', 'LCL-June2015v2_8.csv', 'LCL-June2015v2_9.csv', 'LCL-June2015v2_10.csv', 'LCL-June2015v2_11.csv', 'LCL-June2015v2_12.csv', 'LCL-June2015v2_13.csv', 'LCL-June2015v2_14.csv', 'LCL-June2015v2_15.csv', 'LCL-June2015v2_16.csv', 'LCL-June2015v2_17.csv', 'LCL-June2015v2_18.csv', 'LCL-June2015v2_19.csv', 'LCL-June2015v2_20.csv', 'LCL-June2015v2_22.csv', 'LCL-June2015v2_21.csv', 'LCL-June2015v2_23.csv', 'LCL-June2015v2_24.csv', 'LCL-June2015v2_25.csv', 'LCL-June2015v2_26.csv', 'LCL-June2015v2_27.csv', 'LCL-June2015v2_28.csv', 'LCL-June2015v2_29.csv', 'LCL-June2015v2_30.csv', 'LCL-June2015v2_31.csv', 'LCL-June2015v2_32.csv', 'LCL-June2015v2_33.csv', 'LCL-June2015v2_34.csv', 'LCL-June2015v2_35.csv', 'LCL-June2015v2_36.csv', 'LCL-June2015v2_37.csv', 'LCL-June2015v2_38.csv', 'LCL-June2015v2_39.csv', 'LCL-June

In [ ]:
import pandas as pd

# List to hold DataFrames
df_list = []

# Loop through each file and read into a DataFrame
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    df_list.append(df)

# Concatenate all DataFrames
LCL_df = pd.concat(df_list, ignore_index=True)


In [ ]:
LCL_df

,LCLid,stdorToU,DateTime,KWH/hh (per half hour)
0,MAC000002,Std,2012-10-12 00:30:00.0000000,0
1,MAC000002,Std,2012-10-12 01:00:00.0000000,0
2,MAC000002,Std,2012-10-12 01:30:00.0000000,0
3,MAC000002,Std,2012-10-12 02:00:00.0000000,0
4,MAC000002,Std,2012-10-12 02:30:00.0000000,0
...,...,...,...,...
80999995,MAC003343,Std,2012-09-27 07:00:00.0000000,0.461
80999996,MAC003343,Std,2012-09-27 07:30:00.0000000,0.268
80999997,MAC003343,Std,2012-09-27 08:00:00.0000000,0.149
80999998,MAC003343,Std,2012-09-27 08:30:00.0000000,0.198


In [ ]:
# Strip leading/trailing spaces from column names
LCL_df.columns = LCL_df.columns.str.strip()

# Convert DateTime column to datetime
LCL_df['DateTime'] = pd.to_datetime(LCL_df['DateTime'])

# Convert 'KWH/hh (per half hour)' column to numeric
LCL_df['KWH/hh (per half hour)'] = pd.to_numeric(LCL_df['KWH/hh (per half hour)'], errors='coerce')

In [ ]:
LCL_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81000000 entries, 0 to 80999999
Data columns (total 4 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   LCLid                   object        
 1   stdorToU                object        
 2   DateTime                datetime64[ns]
 3   KWH/hh (per half hour)  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.4+ GB


In [ ]:
LCL_df.isnull().sum()

LCLid                        0
stdorToU                     0
DateTime                     0
KWH/hh (per half hour)    2659
dtype: int64

## Handling Missing Values

In [ ]:
# Drop rows with missing values
LCL_df = LCL_df.dropna()

LCL_df.isnull().sum()

LCLid                     0
stdorToU                  0
DateTime                  0
KWH/hh (per half hour)    0
dtype: int64

## Converting Data Types

In [ ]:
# Convert columns to appropriate data types if necessary
LCL_df['LCLid'] = LCL_df['LCLid'].astype('category')
LCL_df['stdorToU'] = LCL_df['stdorToU'].astype('category')

<ipython-input-14-f1ee69281f09>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LCL_df['LCLid'] = LCL_df['LCLid'].astype('category')
<ipython-input-14-f1ee69281f09>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LCL_df['stdorToU'] = LCL_df['stdorToU'].astype('category')
